<a href="https://colab.research.google.com/github/Clinda02/financial_data_analysis/blob/main/Paxnet_StockReport_Scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

## 증권사 리포트 크롤링
- Paxnet의 증권사리포트의 `종목리포트` 10페이지를 수집하여 데이터 프레임 생성하고, 데이터 프레임을 `report.csv`로 저장
* 사이트 주소 : https://www.paxnet.co.kr/stock/report/report?wlog_rpt=jm&menuCode=2222



**증권사 리포트의 종목리포트 탭**
* `https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo=2&reportId=0&searchKey=stock&searchValue=`
*  페이지네이션이 되어 있고, 페이지 변경시, currentPageNo가 변경됨.

**데이터프레임의 구조**
* `종목명, 제목, 적정가격, 투자의견, 제공출처, 작성일` 6개의 칼럼으로 구성

#  HTML request로 받아오기

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# Paxnet페이지 request 호출
url = "https://www.paxnet.co.kr/stock/report/report?menuCode=2222"
response = requests.get(url)

# 종목 리포트 수집
soup = BeautifulSoup(response.text, 'lxml')

# bs4로 파싱
- 전체 표 : `<div class ="board-type">`
- 리스트 아이템 : `<li>`

In [3]:
# 전체 표
table = soup.find('div', {'class' :'board-type'})

In [4]:
#리스트 아이템
list_items = table.find_all('li')

In [5]:
#리스트 아이템의 첫번째 값 : 컬럼명
list_items[0]

<li class="board-list-th">
<div>종목</div>
<div>제목</div>
<div class="right">적정가격</div>
<div>투자의견</div>
<div>제공출처</div>
<div class="cent">작성일</div>
</li>

In [6]:
#리스트 아이템의 두번째 값
list_items[1]

<li>
<div><strong class="color-cate"><span class="bracket">[</span><a href="http://www.paxnet.co.kr/stock/analysis/main?wlog_rpt=list_jm&amp;abbrSymbol=069960">현대백화점</a><span class="bracket">]</span></strong></div>
<div>
<p>
<a href="javascript:selectView('146055');">3Q24 Preview: 지누스 기대감 솔솔</a>
</p>
</div>
<div class="line3">
<span>적정가격</span>
	 								
										
											65,000원
										
										
									
								</div>
<div class="line3 color-red">
									매수
								</div>
<div class="line3">유진투자증권</div>
<div class="line3">2024.10.31</div>
</li>

In [7]:
#리스트 아이템의 가장 마지막 값
list_items[-5]

<li class="on"><a href="#">1</a></li>

In [8]:
# 종목명
list_items[1].find_all('a')[0].text

'현대백화점'

In [9]:
# 리포트 제목
list_items[1].find_all('a')[1].text

'3Q24 Preview: 지누스 기대감 솔솔'

In [10]:
# 적정가격 <div class ="line3">
list_items[1].find('div', {'class': 'line3'}).text.strip()

'적정가격\r\n\t \t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t65,000원'

In [11]:
# 적정가격 전처리
list_items[1].find_all('div', {'class':'line3'})[0].text.replace('적정가격','').strip()

'65,000원'

In [12]:
# 투자의견 <div class="line3 color-red">
list_items[1].find_all('div', {'class':'line3'})[1].text.strip()

'매수'

In [13]:
# 제공출처 <div class="line3">LS증권</div>

list_items[1].find_all('div', {'class':'line3'})[2].text

'유진투자증권'

In [14]:
# 작성일 <div class="line3">2024.10.28</div>
list_items[1].find_all('div', {'class':'line3'})[3].text

'2024.10.31'

In [15]:
table = soup.find('div',{'class':'board-type'})
list_items = table.find_all('li')

print(f"종목명 : {list_items[1].find_all('a')[0].text}")
print(f"제목 : {list_items[1].find_all('a')[1].text}")
print(f"적정가격 : {list_items[1].find_all('div', {'class' :'line3'})[0].text.replace('적정가격','').strip()}")
print(f"투자의견 : {list_items[1].find('div', {'class' :'line3 color-red'}).text.strip()}")
print(f"제공출처 : {list_items[1].find_all('div', {'class' :'line3'})[2].text.strip()}")
print(f"작성일 : {list_items[1].find_all('div', {'class' :'line3'})[3].text.strip()}")

종목명 : 현대백화점
제목 : 3Q24 Preview: 지누스 기대감 솔솔
적정가격 : 65,000원
투자의견 : 매수
제공출처 : 유진투자증권
작성일 : 2024.10.31


# 페이지네이션을 고려한 반복문 생성

In [16]:
def stock_reports(li):
  table = li.find('div', {'class' :'board-type'})
  list_items = table.find_all('li')
  report_count = len(list_items)

  # 빈 리스트 생성
  stock_name_list = []
  title_list = []
  price_list = []
  opinion_list = []
  trading_firm_list = []
  date_list = []

  for i in range(1,report_count-5):
    stock_name = list_items[i].find_all('a')[0].text  # 종목명
    title = list_items[i].find_all('a')[1].text  # 리포트 제목
    price = list_items[i].find_all('div', {'class':'line3'})[0].text.replace('적정가격','').strip()  # 적정가격
    opinion = list_items[i].find_all('div', {'class':'line3'})[1].text.strip()  # 투자의견
    trading_firm = list_items[i].find_all('div', {'class':'line3'})[2].text  # 제공출처
    date = list_items[i].find_all('div', {'class':'line3'})[3].text  # 작성일

    stock_name_list.append(stock_name)
    title_list.append(title)
    price_list.append(price)
    opinion_list.append(opinion)
    trading_firm_list.append(trading_firm)
    date_list.append(date)

  df = pd.DataFrame({
        '종목명' : stock_name_list
        , '제목' : title_list
        , '적정가격' : price_list
        , '투자의견' : opinion_list
        , '제공출처' : trading_firm_list
        , '작성일' :date_list})

  return df

## 여러 페이지의 정보 가져오기


In [17]:
# 전체 리포트 저장할 데이터프레임 생성
reports_all = pd.DataFrame()

for page_num in range(1,11): # range(1, n+1) : 1부터 n까지
  url = f"https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={page_num}&reportId=0&searchKey=stock&searchValue="
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'lxml')

  reports_all = pd.concat([reports_all, stock_reports(soup)], ignore_index=True)

  # 1초 대기 후 페이지 request 요청
  time.sleep(1)

In [18]:
reports_all.head(2)

,종목명,제목,적정가격,투자의견,제공출처,작성일
0,현대백화점,3Q24 Preview: 지누스 기대감 솔솔,"65,000원",매수,유진투자증권,2024.10.31
1,한화솔루션,서서히 회복 중,"30,000원",매수,유진투자증권,2024.10.31


In [19]:
reports_all.tail(2)

,종목명,제목,적정가격,투자의견,제공출처,작성일
298,JW생명과학,빙빙 돌아가는 수액 공장처럼,15000원,매수(유지),키움증권,2024.09.30
299,JW중외제약,느껴지는 정상화의 흐름,39000원,매수(유지),키움증권,2024.09.30


# CSV 파일로 저장

In [20]:
reports_all.to_csv('report.csv', index=False)
print("csv 파일 저장")

csv 파일 저장
